# Toying with the Lambda Calculus

[John Mount](https://win-vector.com/john-mount/)
May 29, 2025
[Announcement/discussion](https://win-vector.com/2025/05/29/toying-with-the-lambda-calculus/)


## Introduction

There have been a number of great articles on the equational nature of [Lisp](https://en.wikipedia.org/wiki/Lisp_(programming_language)).

  * [Stuart Feldman "A Conversation with Alan Kay"](https://queue.acm.org/detail.cfm?id=1039523)
  * [Peter Norvig "(How to Write a (Lisp) Interpreter (in Python))"](https://norvig.com/lispy.html)
  * [Michael Nielsen "Lisp as the Maxwell’s equations of software"](https://michaelnielsen.org/ddi/lisp-as-the-maxwells-equations-of-software/)

They emphasize how Lisp itself can be considered as a small series of Lisp expressions. These give us a chance to think a bit beyond the imperative von Neumann view (see [John Backus "Can programming be liberated from the von Neumann style?: a functional style and its algebra of programs"](https://dl.acm.org/doi/10.1145/359576.359579)).

A slightly wart is: the relation of Lisps to [the Lambda Calculus](https://en.wikipedia.org/wiki/Lambda_calculus). The grace of Lisps and Lisp implementations is precisely that they do not need to be implemented in terms fo the Lambda Calculus. This is similar to the grace of general purpose computers not needing to be implemented in terms of tape driven [Turing machines](https://en.wikipedia.org/wiki/Turing_machine).

In my opinion, in contrast to a pretty Lisp being implemented in pretty code; the lambda calculus is an ugly language (a [Turing tarpit](https://en.wikipedia.org/wiki/Turing_tarpit)) implemented in terms of a fairly ugly primitive (capture avoiding substitution). Howecer, let's the the lambda calculus out to play anyway.


## The toy engine

We import our toy implementation in Python as a direct set of transforms on tree data structures. This is probably better than working over strings. It is going to be clunkier than translating to [combinatory logic](https://en.wikipedia.org/wiki/Combinatory_logic) implementing over a Lisp, Haskel, or pattern matching re-write engine. And direct manipulation of lambda-expressions is not necessarily a dominant or good idea. There are alternatives to direct evaluation, such as translation to combinators. Some amazing write-ups on these alternative can be found [here](https://crypto.stanford.edu/~blynn/lambda/).

But, let's proceed in a straightforward Python data structure manner. It avoids the need to parse, and is fairly succinct to implement.


In [1]:
from IPython.display import display, HTML, Math, Markdown
import pandas as pd

from lambda_calc import *
load_common_aliases()

def display_def(t):
    display(Math(t.to_latex(not_expanded={t})))


### Expressions

The lambda calculus has only a few primitives:

  * Variable names, such as $x$, $y$, and $f$.
  * Composition in the form of writing one lambda calculus expression $A$ before another $B$ as $A \; B$.
  * Parenthesis. The expressions $(A \; B) \; C$ and $A \; (B \; C)$ are distinct. 
     * For convenience we write $(A\; B) \; C$ as $A \; B \; C$.
     * As writing our examples in Python where $A B$ is not always syntactically allowed we have input notations such as `A | B$` and `v(A, B)`
  * Abstraction in the form of $(\lambda \; x \; . \; A)$. The first portion $\lambda \; x \; .$ names $x$ as a bound variable. The second portion $A$ is called the body. 
     * For example $(\lambda \; x \; . \; x \;y \; x)$ says "$x$ is to be replaced by whatever comes after this abstraction in the body $x \;y \; x$.
     * In Python we will write $(\lambda \; x \; . \; A)$ as `λ[x](A)$`.

More details can be found [here](https://en.wikipedia.org/wiki/Lambda_calculus_definition).

Let's see the abstraction in action. First we declare the example abstraction.


In [2]:
λ['x']('x', 'y', 'x')


λ['x'](('x', 'y'), 'x')

This is read as "prepare to replace the $x$-s in $x \; y \; x$ with something else."


We can add in the "something else" as an expression $z$ to be acted on. 

For our Python powered lambda calculus engine "(A) | (B)" is how we write "A B" without violating Python's syntax rules. One can also use the notation "(A)(B)" to denote "A B". The parenthesis can be omitted if it is known that "A" and "B" are neither of type "composition" (of the form "X Y" themselves). The `v()` function also accepts lists to specify composition.


In [3]:
λ['x']('x', 'y', 'x') | v('z')


v(λ['x'](('x', 'y'), 'x'), 'z')

We can apply a "beta reduction step" using the $.r()$ reduce method. This is the lambda-calculus' version of "'modus ponens": i.e. apply the rule to get the claimed conclusion or trasform.


In [4]:
(λ['x']('x', 'y', 'x') | v('z')).r()


v(('z', 'y'), 'z')

A Python source code text representation of any such expression is available through the `repr()` function. For example.


In [5]:
A = λ['x']('x', 'y', 'x') | v('z')

A


v(λ['x'](('x', 'y'), 'x'), 'z')

In [6]:
str_rep = repr(A)

str_rep


"v(λ['x'](('x', 'y'), 'x'), 'z')"

We can evaluate an expression by either:

  * calling the normal order beta reduction method $.r()$, which runs one substitution step
  * or by calling the normal form method $.nf()$, which repeats $.r()$ until nothing changes.


In [7]:
A.r()


v(('z', 'y'), 'z')

In [8]:
normal_form, steps_taken = A.nf()

normal_form


v(('z', 'y'), 'z')

And we can parse such expressions with `parse_l()` (warning: `parse_l()` does use Python `eval()`).


In [9]:
assert A == parse_l(str_rep)

parse_l(str_rep)


v(λ['x'](('x', 'y'), 'x'), 'z')

### Substitution

The expression $(\lambda \; x \; .\; A) \; B$ is eligible for a re-write called a beta-reduction. In this we replace $(\lambda \; x \; . \; A) \; B$ with $A[x := B]$. $A[x := B]$ denotes collision avoiding substitution where instances of $x$ in $A$ are replaced with $B$, subject to two caveats:

  * Only free instances of $x$ are replaced. "free" means not bound or inside the scope of another abstraction or replacement rule.
  * Substitutions in $A$ must not collide with the names of any containing abstraction variables.

In all cases, conflicts are avoided by re-naming.

Let's look at these difficulties in detail.


The first case is the easy or garden path case, we have no conflicts and substitution is simple.


In [10]:
A = λ['z']('x')
B = v('y')
λ['x']('x')

λ['x'](A) | B


v(λ['x'](λ['z']('x')), 'y')

In [11]:
(λ['x'](A) | B).nf()[0]


λ['z']('y')

Our next case is a bit harder: the variable $x$ in $A$ is bound as the abstraction variable. So it is not "free" or available for substitution. The right answer is: no substitution occurs, just a reduction.


In [12]:
A = λ['x']('x')
B = v('y')
λ['x']('x')

λ['x'](A) | B


v(λ['x'](λ['x']('x')), 'y')

In [13]:
(λ['x'](A) | B).nf()[0]


λ['x']('x')

The final case is perhaps the most subtle one. Substituting $x$ into $y$ would collide with an outer abstraction variable. Such an incorrect substitution would wrongly change the constant function $\lambda \; x \; . \; y$ (that always returns $y$) into the semantically different identity function $\lambda \; x \; . \; x$ (that always returns its argument). To avoid this we must first rename $\lambda \; x \; . \; y$ to $\lambda \; v0 \; . \; y$, and then continue the substitution.

We demonstrate this here.


In [14]:
λ["x"]("y")._capture_avoiding_substitution(
    var=v("y"), 
    t=v("x"), 
    new_name_source=NewNameSource(nms=('x', 'y'))
)


λ['v0']('x')

The above are the ugly issues in realizing the lambda calculus as re-write rules. The substitutions must involve collision avoidance. And this in turn involves re-scanning expressions for bound and unbound variables. The scanning itself is non-local (doesn't depend on a finite number of facts about any one node in the expression) and complicates the implementation and slows down the code. There are alternate variable naming conventions such as [De Bruijn indexing](https://en.wikipedia.org/wiki/De_Bruijn_index) that mitigate this issue, however even these symbols require non-constant sized context for interpretation.

Normal form substitution involves choosing the top left-most reduction. Again, that is a non-local property of the expression.

This raises the philosophical question of the lambda-calculus is *really* a set of local operations (like the operation of the state-head of a Turing machine is).


## Data types

The basic lambda calculus doesn't have *any* data types. Data types are simulated by building expressions that work as the data would.

This is largely through known conventions. Certain expressions are taken to represent specific values. For example


In [15]:
forms = [TRUE, FALSE, N(0), N(1), N(2), N(3)]
display(HTML(pd.DataFrame({
    'named form': [str(v) for v in forms],
    'Python representation': [repr(v) for v in forms],
}).to_html()))


,named form,Python representation
0,TRUE,λ['x'](λ['y']('x'))
1,FALSE,λ['x'](λ['y']('y'))
2,N(0),λ['f'](λ['x']('x'))
3,N(1),"λ['f'](λ['x']('f', 'x'))"
4,N(2),"λ['f'](λ['x']('f', ('f', 'x')))"
5,N(3),"λ['f'](λ['x']('f', ('f', ('f', 'x'))))"


The convention is: when you see $\lambda \; f \;.\; ( \lambda \; x \;.\; f \; (f \; x))$ (possibly with variable name substitutions), you are supposed to think "oh, that is a 2".


## Data as functions

One of the tricks used to make the lambda calculus work is exploiting what expression is used to represent a given datum. 


### TRUE and FALSE

The representations of TRUE and FALSE are picked so that TRUE selects the first thing after it and FALSE selects the second. This is easiest to see by example.


In [16]:
expr = TRUE | 'a' | 'b'

expr


v((λ['x'](λ['y']('x')), 'a'), 'b')

In [17]:
expr.nf()[0]


v('a')

In [18]:
expr = FALSE | 'a' | 'b'

expr


v((λ['x'](λ['y']('y')), 'a'), 'b')

In [19]:
expr.nf()[0]


v('b')

### The Church numerals

The Church numerals are picked to have useful properties. Tellingly: `0` is equivalent to `FALSE`. They both select their second following term.


In [20]:
(N(0) | 'a' | 'b').nf()[0]


v('b')

`1` is not the same as `TRUE`. Instead we have `(k a b).nf()[0]` is `a` applied to `b` `k` times.


In [21]:
(N(3) | 'a' | 'b').nf()[0]


v('a', ('a', ('a', 'b')))

There is a traditional expression called `ISZERO` used to convert arbitrary Church numerals into a selector. 


In [22]:
display_def(ISZERO)


<IPython.core.display.Math object>

In [23]:
(ISZERO | N(0) | 'a' | 'b').nf()[0]


v('a')

In [24]:
(ISZERO | N(3) | 'a' | 'b').nf()[0]


v('b')

Some study will show `ISZERO` works in the following sequence:

  * Copying the number to be compared to the left so it would be next to be executed in normal order.
  * Expanding the Church humber on the left to get `k` copies of `(λx.FALSE)` applied followed by one copy of `TRUE`.
    * If `k` is zero then that is just a `TRUE`.
    * If `k` is greater than zero the `(λx.FALSE)`s plus `TRUE` collapses to a single `FALSE`.
  * This remaining `FALSE` or `TRUE` is then applied to the next two arguments to select the first or second.


The above exposes a key point on the lambda calculus. Code written in it is not referentially transparent. The exact code depends on the exact function used to represent things. We can't replace the numerals with other representations without re-writing manu functions such as ISZERO. Also the language is very typeless. It is impossible to say if a variable holds a value, a function, or something in-between. Using functions to represent data seems harmless. But it entails the very constraining use of data representations as functions.


## Additional functions

We can define some additional functions such as predecessor.


In [25]:
display_def(PRED)


<IPython.core.display.Math object>

Let's see predecessor in action


In [26]:
(PRED | N(3)).nf()[0]


λ['f'](λ['x']('f', ('f', 'x')))

Let's see that again slowed down a bit.


In [27]:
def run_beta_reduction(t: Term) -> Term:
    display(Math(t.to_latex()))
    last_t = None
    while True:
        t = t.r()
        if t == last_t:
            return t
        display(Math(t.to_latex()))
        last_t = t


In [28]:
run_beta_reduction(PRED | N(3))


<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

...

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

λ['f'](λ['x']('f', ('f', 'x')))

It requires a great number of intermediate steps to get from 3 to 2.


## Combinators

We have not yet defined how to apply an operation more than a constant number of times. That is what we will now pursue.

To get repetition, iteration, or recursion we need a trick called "combinators." These are operators that copy sections of lambda expressions to implement recursion and iteration.


### The Y combinator

The most famous combinator is the Y combinator.


In [29]:
display_def(Y)


<IPython.core.display.Math object>

It works like this.


In [30]:
t = Y | v('a')

t


v(λ['f'](λ['x']('f', ('x', 'x')), λ['x']('f', ('x', 'x'))), 'a')

A first reduction step specializes Y by capturing the argument "a".


In [31]:
t =  t.r()

t


v(λ['x']('a', ('x', 'x')), λ['x']('a', ('x', 'x')))

From then on repeated reductions create copies of "a" on the left.


In [32]:
t = t.r()

t


v('a', (λ['x']('a', ('x', 'x')), λ['x']('a', ('x', 'x'))))

In [33]:
t = t.r()

t


v('a', ('a', (λ['x']('a', ('x', 'x')), λ['x']('a', ('x', 'x')))))

### The Θ combinator

Turing defined a similar combinator called "Θ". It reproduces its argument, like the Y combinator, but doesn't specialize.


In [34]:
# Theta combinator
# https://www.pls-lab.org/en/Theta_combinator
# Θ≜(λxy.y(xxy))(λxy.y(xxy))


display_def(Θ)


<IPython.core.display.Math object>

In [35]:
t = Θ | v('a')

t


v((λ['x'](λ['y']('y', (('x', 'x'), 'y'))), λ['x'](λ['y']('y', (('x', 'x'), 'y')))), 'a')

Every two reduction steps the Θ reconstructs itself and copies its argument to the left.


In [36]:
t = t.r()

t


v(λ['y']('y', ((λ['x'](λ['y']('y', (('x', 'x'), 'y'))), λ['x'](λ['y']('y', (('x', 'x'), 'y')))), 'y')), 'a')

In [37]:
t = t.r()

t


v('a', ((λ['x'](λ['y']('y', (('x', 'x'), 'y'))), λ['x'](λ['y']('y', (('x', 'x'), 'y')))), 'a'))

Now, as normal form reductions apply to the top and left of the expression tree, if $a$ had any potential reductions they would be applied before the combinator fires again. Ideas such as "move left to delegate" and "copy to iterator or recurse" are central to designing procedures in the lambda calculus.


## Iterating

We can use either of the combinators to implement an iterative algorithm such as [factorial](https://en.wikipedia.org/wiki/Factorial).


In [38]:
# https://jwodder.freeshell.org/lambda.html
display_def(FACTORIALstep)


<IPython.core.display.Math object>

`FACTORIALstep` is the most complicated expression we have yet defined. It captures the combinator in `g` and the factorial argument in `x`. `ISZERO` either returns the base case `1` (throwing away the combinator and ending the calculation), or expands the `g (PRED x)` argument, applying the function and combinator to the predecessor of `x`.

Let's run an example.


In [39]:
t = Y | FACTORIALstep | N(4)

t


v((λ['f'](λ['x']('f', ('x', 'x')), λ['x']('f', ('x', 'x'))), λ['g'](λ['x'](((λ['n'](('n', λ['x'](λ['x'](λ['y']('y')))), λ['x'](λ['y']('x'))), 'x'), λ['f'](λ['x']('f', 'x'))), ((λ['m'](λ['n'](λ['f'](λ['x'](('m', ('n', 'f')), 'x')))), 'x'), ('g', (λ['n'](λ['f'](λ['x']((('n', λ['g'](λ['h']('h', ('g', 'f')))), λ['u']('x')), λ['u']('u')))), 'x')))))), λ['f'](λ['x']('f', ('f', ('f', ('f', 'x'))))))

Combining this code computes the factorial function.


In [40]:
t.nf()[0]


λ['f'](λ['x']('f', ('f', ('f', ('f', ('f', ('f', ('f', ('f', ('f', ('f', ('f', ('f', ('f', ('f', ('f', ('f', ('f', ('f', ('f', ('f', ('f', ('f', ('f', ('f', 'x')))))))))))))))))))))))))

And that is the right answer.

Again, we are using the normal beta reduction order: always using the highest left-most available reduction. We will talk on reduction roder a bit more later.

Let's count how many steps this took.


In [41]:
res, steps = t.nf()

res


λ['f'](λ['x']('f', ('f', ('f', ('f', ('f', ('f', ('f', ('f', ('f', ('f', ('f', ('f', ('f', ('f', ('f', ('f', ('f', ('f', ('f', ('f', ('f', ('f', ('f', ('f', 'x')))))))))))))))))))))))))

In [42]:
{'steps': steps}


{'steps': 3914}

It took a lot of steps to get to 24.


## Reduction order

The reduction rule we are using is the normal order: always top left most available beta-reduction. This is the "lazy evaluation" order where arguments that are not used are not expanded. The advantage of normal reduction order is it can throw away poison arguments such as in the following. The disadvantage is arguments are passed around unevaluated, so the grow and they get re-computed when re-used.


## Deterministic calculation

Not all lambda expressions reduce to a normal form. Some iterate forever. So grow, some oscillate, and some have arbitrarily complicated behavior. In fact determining if a form reduces to a normal form is undecidable.

A typical such poison form is the following (in analogy to [Conway's Game of Life](https://en.wikipedia.org/wiki/Conway%27s_Game_of_Life) blinker).


In [43]:
a = λ['x', 'y']('y', 'x', 'y')
blinker = a | a | a

blinker


v((λ['x'](λ['y'](('y', 'x'), 'y')), λ['x'](λ['y'](('y', 'x'), 'y'))), λ['x'](λ['y'](('y', 'x'), 'y')))

In [44]:
assert blinker.r() != blinker

blinker.r()


v(λ['y'](('y', λ['x'](λ['y'](('y', 'x'), 'y'))), 'y'), λ['x'](λ['y'](('y', 'x'), 'y')))

In [45]:
assert blinker.r().r() == blinker

blinker.r().r()


v((λ['x'](λ['y'](('y', 'x'), 'y')), λ['x'](λ['y'](('y', 'x'), 'y'))), λ['x'](λ['y'](('y', 'x'), 'y')))

For substitutions that do stabilize, the Church–Rosser theorem states we can get to a canonical ending form by applying the beta-reductions in normal oder. Which is what we do. This establishes two things:

  1) If any sequence terminates, then the normal order one does.
  2) All terminating expressions are equivalent.

What this is saying: the re-writing rules are safe to use.

An example of an unsafe rule set would be the following two rules:

  1) For a tuple "A B" delete the left item.
  2) For a tuple "A B" delete the right item.

One rules finalizes "`A B`" to "`B`", and the other finalizes  "`A B`" to "`A`". This produces different answers depending execution order. The lambda calculus does not have this flaw. If the evaluation halts, then it halts in single form (up to some name equivalences). This is called "confluence" in evaluation. Thus we know the above two rule system can not be implemented in the lambda calculus.

A key benefit of the lambda calculus is: if one can argue another more powerful evaluation system *could* be implemented by the lambda calculus, then that system is also confluent. We don't need the implementation to know that, we just need to know it *could* be implemented.


## Encodings

How one encodes a lambda expression has been of central interest to researchers. In particular there is great interest in the circularity of such representations.

The lamba calculus can represent:

  * Non-negative integers by the Church numerals.
  * Lists by associative construction.
  * Strings by sequential lists.
  * Binary strings through sequences of `TRUE`/`FALSE`.
  * Combinators through replacement rules.

Conversely the following systems can encode the lambda calculus.

  * Non-negativer integers by Gödel encoding (see Alonzo Church, "An Unsolvable Problem of Elementary Number Theory", American Journal of Mathematics, Vol. 58, No. 2. (Apr., 1936), pp. 345-363.). The lambda calculus needs an infinite set for representation, and the non-negative integers are considered to be among the simplest such sets.
  * Lists/trees: such as Lisp's `car`/`cdr` system or the demonstrated tree of Python objects.
  * Strings. To us this is obvious in the sense we expect programs to have source code. However this opinion likely derives from institutional experience with the lambda calculus.
  * Binary strings, which are a more familiar structure in the digital computer age. An amazing variation of the lambda calculus is the John Tromp's [binary lambda calculus](https://tromp.github.io/cl/Binary_lambda_calculus.html). Links:
    * [esolangs, "Binary lambda calculus"](https://esolangs.org/wiki/Binary_lambda_calculus#self-interpreter)
    * [Tromp, "Binary Lambda Calculus"](https://tromp.github.io/cl/Binary_lambda_calculus.html)
    * [John's Combinatory Logic Playground](https://tromp.github.io/cl/cl.html)
    * [blc interpeter](https://github.com/tromp/AIT/blob/master/uni.c)
  * [Combinator calculus](https://en.wikipedia.org/wiki/SKI_combinator_calculus) via abstraction elimination.


## Conclusion

It is always seems a surprise how many steps were needed to execute examples. Part of that is the primitive nature of the operations and a great part of that is consequence of lazy evaluation.

The lambda calculus was used by Church and others to simplify examples of undecidable assertions about calculation and decisions. The embeddings of the lambda calculus into other systems the simplifies demonstrating undecidable assertions in those systems. So it should not be too shocking that the lambda calculus is difficult to evaluate, it was designed to capture difficulty.

Programs are useful because a large subset of them do in fact terminate with useful results. The lambda calculus is not a good encoding for these useful programs, it is more useful for discussing common pathological cases in terms of simple primitives.

Links:

  * [Announcement/discussion](https://win-vector.com/2025/05/29/toying-with-the-lambda-calculus/)
  * Source code [lambda_calc.py](lambda_calc.py)
  * Unit test [tests/test_lambda.py](tests/test_lambda.py)
  * GCD exampe [div.ipynb](div.ipynb).

